<p></p><font size=8 face="courier" color="magenta">Visualize</font>

This notebook creates the widget inside of the "Visualize" section of the [Cytoscape app](../Cytoscape.ipynb).

# Setup Notebook

In [280]:
""" DEBUG  
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug
# %cd sets the path to that of Cytoscape.ipynb
"""

/root/Documents


The **<font color="magenta">create</font>** object holds all the code for this notebook

## Helpers

These are <font color="blue">functions</font> and <font color="magenta">classes</font> that help the visualizer do it's thing.

# Create <font color="magenta">Visualizer</font>

Here we create the initial <font color="magenta">Visualizer</font> class. This class is used to creat and manage the networks visualization. This section only consists of the initialization <font color="magenta">Visualizer</font> and some of it's basic functions. Most of <font color="magenta">Visualizer's</font> functionality is actually implemented inside of the **Add Widgets** section

## <font color="magenta">Visualizer</font> Class

The <font color="magenta">Visualizer</font> Class is what is used to create and manage the networks visualization

In [38]:
class Graph():
    def __init__(self):
        #self.w = type("widgets",(),{})() #<-- this is where all the widgets go
        pass

##  <font color="blue">Functions</font>

###  <font color="blue">canvasHtml</font>

<font color="blue">canvasHtml</font> creates the html that the network will be displayed in

 <font color="red">made width it 50 just for debugging</font>

In [39]:
def canvasHtml(self): 
    display(HTML("""
    <style> .cy { width: 50vw; height: 30vw}</style> 
    <div id="cy" class="cy"></div>
    """))
Graph.canvasHtml = canvasHtml

# <font color="orange">Javascript</font> Helpers

In [40]:
class JS_Helper():
    pass
Graph.js = JS_Helper()

## <font color="orange">addNode</font>

This functions adds new nodes to the network when the clicks and empty part of the graph.

In [42]:
def addNode(self):
    JS("""
    cy.io.addNode = function(event){
      if( event.target === cy ){                                                          // <-- If we tapped on the background
          checkit = cy.add([ {  group: "nodes", position: event.position  } ]);          //<-- add node    
      }; 
    };
    cy.on('tap', cy.io.addNode );
    """)
JS_Helper.addNode = addNode

## <font color="orange">addEdge</font>

We add a new edge when the use double clicks one nodes and then double clicks another node. <font color="orange">addEdge</font> check for the double clicking and adds a new edge if necessary.

In [43]:
def addEdge(self):
    JS("""
    cy.io.click_hist = ["1", "2", "3", "4"];
    
    cy.io.addEdge = function(event){                                    // <-- on "tapping" canvas
        cy.io.click_hist.shift();                                         //<-- Remove oldest click

        if( event.target !== cy && event.target.isNode() ){               // <-- If we tapped a Node
          cy.io.click_hist.push( event.target.id() );                   // <-- Add node id to click history

          if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){      //<-- If 2 double clicks
              cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]);         //<-- add edge
          };

        }else{                                                 //<-- if we didn't click a node
          cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
        }; 
    };
    cy.on("click" , cy.io.addEdge );
    """)
JS_Helper.addEdge = addEdge

### <font color="orange">start</font>

<font color="orange">start</font> makes it possible to edit the canvas that displays the network and loads in the <font color="orange">javascript</font> helpers

In [101]:
def start(self):
    JS("""
    var cy;
    require(['cytoscape'], function(cytoscape){

        cy = cytoscape({ container: document.getElementById('cy'), 
                        layout: { name: 'cose' }
            });
        cy.io = {};   //<--  Where we store all our javascript variables
    });
    """)
JS_Helper.start = start

#  Add <font color="green">Widgets</font>


In [45]:
Graph.w = type("widget_list",(),{})()
Graph.wf = type("widget_functions",(),{})()

## <font color="green">Control Panel</font> 

**<font color="green">widget : </font>**

In [121]:
Graph.w.ControlPanel= widgets.ToggleButtons(
    options=[ 'Edit','Search','Graph Style'])
#Graph.w.ControlPanel

**<font color="blue">Function : </font>**  <font color="blue">ControlChanged</font> changes the editorBox display when the Control Panel changes.

In [122]:
def ControlChanged(b):
    #widg = Graph.w.ControlPanel.value.replace(" ", "")
    #Graph.editor_box.widg = getattr(Graph.w, widg).show
    #Graph.editor_box.execute()
    #clear_output()
    #display(Graph.w.ControlPanel)
    pass

    
Graph.w.ControlPanel.observe( ControlChanged )

How to change button style

In [58]:
# Graph.w.ControlPanel.value = "Search"

## <font color="green">Edit</font>  

In [213]:
Graph.w.Edit = type("widget",(),{})()

### Widgets

#### Delete Button

**<font color="green">widget : </font>**

In [223]:
Graph.w.Edit.delete = ipw.Button(description='Delete',button_style='danger')
#Graph.w.Edit.delete

**<font color="blue">Function : </font>** 

In [224]:
def delete(b):
    JS("cy.io.last_deleted = cy.$(':selected').remove();")   #<-- delete elements
    Graph.w.Edit.undo.disabled = False
Graph.w.Edit.delete.on_click( Graph.wf.delete )

#### Undo Delete 

**<font color="green">widget : </font>**

In [225]:
Graph.w.Edit.undo = ipw.Button(description='Undo',button_style='primary', disabled= True )
#Graph.w.Edit.undo

**<font color="blue">Function : </font>** 

In [226]:
def undo(b):
    JS(" cy.add( cy.io.last_deleted );")   #<-- Add last deleted element
    b.disabled = True
Graph.wf.undo = undo
Graph.w.Edit.undo.on_click( Graph.wf.undo )

#### Color

**<font color="green">widget : </font>**

In [227]:
Graph.w.Edit.color = ipw.ColorPicker( description='Color :' )
Graph.w.Edit.color

ColorPicker(value='black', description='Color :')

**<font color="blue">Function : </font>** 

In [228]:
def color(b):
    #JS(" cy.add( cy.io.last_deleted );")   #<-- Add last deleted element
    return
Graph.wf.color = color
Graph.w.Edit.color.observe( Graph.wf.color )

### Display Boxes

In [229]:
Graph.w.Edit.boxAll = ipw.VBox([ Graph.w.Edit.color,
                                ipw.HBox([ Graph.w.Edit.delete, Graph.w.Edit.undo ])
                               ])

In [231]:
#Graph.w.Edit.boxAll

** What I previously wrote: **


The editor box is the thing next to the visualizer where a user can change the graph. Notice that the contents of this box changes depending on what the user clicks on the control panel. The Control Panel/ Editor Box functionality is actually a little tricky to create in jupyter and require some special things to be built.


## Helper: <font color="magenta">JupyterCell</font>

<p></p>
 <font color="magenta">JupyterCell</font> is a useful class that allows you to execute a specified *jupyter cell* from the command <font color="magenta">JupyterCell</font><font color="blue">.execute()</font>. This is used to change which <font color="green">widget</font> is displayed inside of the visualizer's **editor cell**.

In [212]:
class JupyterCell():
 
    def __init__(self, cell_text, text_before="# "):
        self.cell_text= cell_text
        self.text_before= text_before
    
    def execute(self):  #<-- This gets the Cell by using the cell text. The cell text must be unique to that cell
        JS("""
        var cell_text= "";  var i= 0;  
        while( ! cell_text.includes('"""+self.text_before+self.cell_text+"""')  ){
            cell_text = Jupyter.notebook.get_cell( i ).get_text()  //.element["0"].innerText #<-- Old Way
            i++
        }
        IPython.notebook.execute_cells([i-1])
        """)


## Create Editor  Box

In [278]:
Graph.editor_box = JupyterCell("My Edit Box")
Graph.editor_box.widg = ipw.HTML("<h5> </h5>") #<-- widget initially empty

Graph.editor_box.execute()

In [271]:
def testit():
    Graph.editor_box.widg = Graph.w.Edit.boxAll
    Graph.editor_box.execute()

In [272]:
testit()

In [262]:
JS("""Jupyter.notebook.kernel.execute("Graph.editor_box.execute()")""")

# Visualize (Mock Setup)

In [281]:
cy = Graph()

In [282]:
# My Edit Box
display( Graph.w.ControlPanel )
display( Graph.editor_box.widg  )



ToggleButtons(options=('Edit', 'Search', 'Graph Style'), value='Edit')

In [283]:
cy.canvasHtml()
cy.js.start()

In [284]:
cy.js.addNode()
cy.js.addEdge()

In [277]:
JS("""
    var check = 0;
    var check_var;

    cy.io.test = function(event){
        cy.io.selection = cy.$(':selected')
        
        if( cy.io.selection.length == 0  ){                 //<-- Do nothing if nothing's selected
            return
        } else if( cy.io.selection.length == 1 ){           //<-- If selection lenght is 1
        
            if( cy.io.selection.isNode() ){                      //<-- If it's a node
                console.log( "1 node" )
                Jupyter.notebook.kernel.execute("testit()")
                //Jupyter.notebook.kernel.execute("Graph.editor_box.execute() ")
                
            } else{                                              //<-- If it's an edge
                console.log( "1 edge" )
            }
            
        } else {                                           //<-- If Many elements
            console.log("many items")
        }
        
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")


In [276]:
JS("""
    cy.removeListener('select unselect', cy.io.test);
""")

In [ ]:
cy.$(':selected')

checkit[1].isNode()
checkit[0].isEdge()



Old test function

In [ ]:
JS("""
    var check = 0;
    var check_var;

    cy.io.test = function(event){
        cy.io.selection = cy.$(':selected')
        
        check += 1
        console.log( String(check) +": selection changed")
        check_var = 
        cehck_var2 = event
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")


In [70]:
#import inspect
#lines = inspect.getsource(cy.js.addEdge)
#print(lines)

# Scratch Paper

In [90]:
JS("""
//cy.removeListener('tap', handler);
//cy.removeListener( "click" ) 
//cy.removeListener( "tap" )
//cy.removeListener( "select" )
//cy.removeListener( "unselect" )
""")

## Older stuff

In [75]:
"""
JS(
cyto.add([
  { group: "nodes", data: { id: "n0" } , position: { x: 100, y: 200 } },
  { group: "nodes", data: { id: "n1" }, position: { x: 200, y: 100 } },
  { group: "edges", data: { id: "e0", source: "n0", target: "n1" } }
]);
)"""


'\nJS(\ncyto.add([\n  { group: "nodes", data: { id: "n0" } , position: { x: 100, y: 200 } },\n  { group: "nodes", data: { id: "n1" }, position: { x: 200, y: 100 } },\n  { group: "edges", data: { id: "e0", source: "n0", target: "n1" } }\n]);\n)'

In [76]:
"""
JS(

cyto.layout({ name: 'circle'}).run();

)
"""

"\nJS(\n\ncyto.layout({ name: 'circle'}).run();\n\n)\n"

In [77]:
"""cy.layout({ name: 'circle'}).run();"""

"cy.layout({ name: 'circle'}).run();"